# Notebook for Multi-label classification training routine 

In [1]:
import os
import json
import re
import string
import random
import time
import datetime

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from argparse import Namespace
from tqdm.notebook import tqdm

# from datasets import Dataset

import transformers
from transformers import BertTokenizer, BertModel, BertConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import pipeline
from transformers import BertTokenizer, DataCollatorForLanguageModeling

import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Dataset

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.preprocessing import LabelEncoder

2024-03-24 16:37:02.609442: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 16:37:03.496005: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
args = Namespace(
    data_path = './processed_data/ir_data.csv',
    base_model_path = "bert-base-uncased",
    model_save_path="mcl_model",
    num_samples=15000,
    train_split=0.7,
    epochs=5,
    batch_size=16,
    patience=3,
    freeze=False,
    learn_rate = 3e-5
)

## Data Preparation

In [3]:
df = pd.read_csv(args.data_path)[0:args.num_samples]

In [4]:
df['lab'] = 1

In [5]:
tokenizer = BertTokenizer.from_pretrained('casehold/legalbert')

In [6]:
df['split'] = 'train'
num_train_rows = int(len(df) * (1 - args.train_split)//2) - 1
df.loc[:num_train_rows, 'split'] = 'val'
df.loc[num_train_rows:num_train_rows + num_train_rows, 'split'] = 'test'
df.head()

,Unnamed: 0,provision,label,source,lab,split
0,0,Section and Subsection headings in this Amendm...,headings,2019/QTR1/000119312519044328/d691151dex101.htm,1,val
1,1,THIS AMENDMENT AND THE RIGHTS AND OBLIGATIONS ...,applicable laws,2019/QTR1/000119312519044328/d691151dex101.htm,1,val
2,2,This Amendment may be executed in any number o...,counterparts,2019/QTR1/000119312519044328/d691151dex101.htm,1,val
3,3,The execution and delivery of this Amendment b...,binding effects,2019/QTR1/000119312519044328/d691151dex101.htm,1,val
4,4,Each of the parties hereto irrevocably waives ...,waiver of jury trials,2019/QTR1/000119312519044328/d691151dex101.htm,1,val


In [7]:
label_encoder = LabelEncoder()

In [8]:
####################################################
############## Setup Train Dataloader ##############
####################################################

encoded_data_train = [tokenizer.encode_plus(row['provision'], row['label'], add_special_tokens=True, max_length=512, pad_to_max_length=True, truncation=True) for index,row in df[df['split'] == 'train'].iterrows()]
input_ids_train = [item['input_ids'] for item in encoded_data_train]
attention_masks_train = [item['attention_mask'] for item in encoded_data_train]
labels_train = [row['lab'] for index,row in df[df['split'] == 'train'].iterrows()]

# Convert to tensors
input_ids_train = torch.tensor(input_ids_train)
attention_masks_train = torch.tensor(attention_masks_train)
labels_train = torch.tensor(labels_train)

# Create a dataset
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)

dataloader_train = DataLoader(dataset_train, batch_size=args.batch_size, shuffle=True) # NOTE : maybe set pin_memory=True

/home/jz75/Documents/2023-2024/SH-Project/CS4099-LegalNLP/pytorchenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [9]:
####################################################
############## Setup Val Dataloader ################
####################################################

encoded_data_val = [tokenizer.encode_plus(row['provision'], row['label'], add_special_tokens=True, max_length=512, pad_to_max_length=True, truncation=True) for index,row in df[df['split'] == 'val'].iterrows()]
input_ids_val = [item['input_ids'] for item in encoded_data_val]
attention_masks_val = [item['attention_mask'] for item in encoded_data_val]
labels_val = [row['lab'] for index,row in df[df['split'] == 'val'].iterrows()]

# Convert to tensors
input_ids_val = torch.tensor(input_ids_val)
attention_masks_val = torch.tensor(attention_masks_val)
labels_val = torch.tensor(labels_val)

# Create a dataset
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

dataloader_val = DataLoader(dataset_val, batch_size=args.batch_size, shuffle=True) # NOTE : maybe set pin_memory=True

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [10]:
###################################################
############## Setup Test Dataloader ##############
###################################################

encoded_data_test = [tokenizer.encode_plus(row['provision'],row['label'], add_special_tokens=True, max_length=512, pad_to_max_length=True, truncation=True) for index,row in df[df['split'] == 'test'].iterrows()]
input_ids_test = [item['input_ids'] for item in encoded_data_test]
attention_masks_test = [item['attention_mask'] for item in encoded_data_test]
labels_test = [row['lab'] for index,row in df[df['split'] == 'test'].iterrows()]


# Convert to tensors
input_ids_test = torch.tensor(input_ids_test)
attention_masks_test = torch.tensor(attention_masks_test)
labels_test = torch.tensor(labels_test)

# Create a dataset
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

dataloader_test = DataLoader(dataset_test, batch_size=args.batch_size, shuffle=True) # NOTE : maybe set pin_memory=True

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

# Training

In [11]:
# Initialize model and optimizer

# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # num_labels = 2 for binary classification
model = BertForSequenceClassification.from_pretrained(args.base_model_path)

if args.freeze: 
    for param in model.base_model.parameters():
        param.requires_grad  = False
    # model.classifier = torch.nn.Linear(model.config.hidden_size, 2)



if args.freeze: 
    optimizer = AdamW(model.classifier.parameters(), lr=args.learn_rate)
else : 
    optimizer = AdamW(model.parameters(), lr=args.learn_rate)

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=len(dataloader_train) * 0.05, num_training_steps=len(dataloader_train) * args.epochs)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/jz75/Documents/2023-2024/SH-Project/CS4099-LegalNLP/pytorchenv/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()#.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100


def calculate_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [13]:
torch.cuda.empty_cache()
if torch.cuda.is_available():
  args.device = 'cuda'

model.to(args.device)
print(args.device)

cuda


In [14]:
train_progress = tqdm(total=0, desc='Train Batches', leave=True)
validation_progress = tqdm(total=0, desc='Validation Batches', leave=True)
epoch_progress = tqdm(total=args.epochs, desc='Epoch', leave=True)

best_val_accuracy = 0.0
patience = 3
num_epochs_no_improvement = 0


loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(args.epochs):

  model.train()
  total_train_loss = 0
  total_train_accuracy = 0 #NEW

  train_progress.reset(total=len(dataloader_train))
  validation_progress.reset(total=len(dataloader_val))

  for step, batch in enumerate(dataloader_train):
    b_input_ids, b_input_mask, b_labels = batch
    b_input_ids, b_input_mask, b_labels = b_input_ids.to(args.device), b_input_mask.to(args.device), b_labels.to(args.device)

    model.zero_grad()
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

    loss = outputs.loss
    total_train_loss += loss.item()
    loss.backward()

    logits = outputs.logits.detach().cpu().numpy()#NEW
    label_ids = b_labels.to('cpu').numpy()#NEW
    total_train_accuracy += calculate_accuracy(logits, label_ids)#NEW

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()

    train_progress.update(1)

  avg_train_loss = total_train_loss / len(dataloader_train)
  print(f'Epoch {epoch}: Average Training Loss: {avg_train_loss}')

  model.eval()
  total_eval_accuracy = 0
  total_eval_loss = 0

  for batch in dataloader_val:
    b_input_ids, b_input_mask, b_labels = batch
    b_input_ids, b_input_mask, b_labels = b_input_ids.to(args.device), b_input_mask.to(args.device), b_labels.to(args.device)

    with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    total_eval_loss += loss_fn(outputs.logits.squeeze(-1), b_labels).item() # perhaps just outputs.loss (need to include labels as parameter in model() above)

    total_eval_accuracy += calculate_accuracy(logits, label_ids)

    validation_progress.update(1)

  avg_val_accuracy = total_eval_accuracy / len(dataloader_val)
  print(f'Epoch {epoch}: Validation Accuracy: {avg_val_accuracy}')

  # Checkpointing and Early Stopping
  if avg_val_accuracy > best_val_accuracy:
      print(f'Validation accuracy improved from {best_val_accuracy} to {avg_val_accuracy}. Saving model...')
      best_val_accuracy = avg_val_accuracy
      num_epochs_no_improvement = 0
      # Save the model using save_pretrained
      model.save_pretrained(args.model_save_path)
  else:
      num_epochs_no_improvement += 1
      if num_epochs_no_improvement >= args.patience:
          print("Early stopping triggered.")
          break  # Exit the training loop

  epoch_progress.update(1)


  

Train Batches: 0it [00:00, ?it/s]

Validation Batches: 0it [00:00, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 0: Average Training Loss: 0.014987350864307279
Epoch 0: Validation Accuracy: 1.0
Validation accuracy improved from 0.0 to 1.0. Saving model...
Epoch 1: Average Training Loss: 3.124188103108781e-05
Epoch 1: Validation Accuracy: 1.0
Epoch 2: Average Training Loss: 1.5196360570954121e-05
Epoch 2: Validation Accuracy: 1.0
Epoch 3: Average Training Loss: 1.0138834576657628e-05
Epoch 3: Validation Accuracy: 1.0
Early stopping triggered.


## Evaluation

In [15]:
from transformers import BertTokenizer, BertModel, BertConfig
def evaluate_sequence_pair_class(model_path,  title):
    '''
    Routine for evaluating model for sequence pair classification
    '''

    progress = tqdm(total=len(dataloader_test), desc='Train Batches', leave=True)

    # load model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    # model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)


    # Check if cuda available
    if torch.cuda.is_available():
        # model.to('cuda')
        args.device = 'cuda'
    else:
        # model.to('cpu')
        args.device = 'cpu'

    print(args.device)

    model.to(args.device)

    model.eval()

    predictions, true_labels = [], []

    print('Evaluating ' + f'[{title}]')
    print('============================================')


    with torch.no_grad(): # disable calculating gradients (more efficient for evaluation)
        for batch in dataloader_test:
            progress.update(1)

            input_ids, attention_mask, labels = tuple(t.to(args.device) for t in batch)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).flatten() # find index of max value in logits tensor (where each index corresponds to a binary class)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    print(len(set(true_labels)))
    print(len(set(predictions)))

    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, acc = precision_recall_fscore_support(true_labels, predictions, average='binary')
    print(acc)
    print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}')



In [16]:
evaluate_sequence_pair_class(args.model_save_path,  'MCL Base Model Evaluation Metrics')

Train Batches:   0%|          | 0/141 [00:00<?, ?it/s]

cuda
Evaluating [MCL Base Model Evaluation Metrics]
1
1
None
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [17]:
evaluate_sequence_pair_class('./models/combined/parallel_combined',  'MCL Base Model Evaluation Metrics')

Train Batches:   0%|          | 0/141 [00:00<?, ?it/s]

cuda
Evaluating [MCL Base Model Evaluation Metrics]
1
2
None
Accuracy: 0.9977777777777778
Precision: 1.0
Recall: 0.9977777777777778
F1 Score: 0.9988876529477196


: 